In [1]:
"""Scrapes https://www.bundestag.de/services/opendata for all XML Files of the 19th and 20th election period."""

import requests
import re
import time
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [7]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import xml.sax
import pandas as pd

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--profile-directory=Profile2")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--headless")

url_20_wahlprediode = "https://www.bundestag.de/ajax/filterlist/de/services/opendata/866354-866354?limit=10&noFilterSet=true&offset=20"
url_19_wahlprediode =""

url = "https://www.bundestag.de/services/opendata"
chromedriver = "/usr/bin/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

#driver = webdriver.Chrome(chromedriver, options=chrome_options) #deprecation warning: pass Service object

In [4]:
#driver.get(url)

link_list = []

# fügt alle gefundenen Links einer Liste hinzu
def add_to_link_list():
    soup_links = BeautifulSoup(driver.page_source, 'html.parser')
    regex = re.compile('.*XML .*')
    documents = soup_links.find_all(title=regex)
    for i in documents:
        link_list.append(i['href'])
    
#for i in range(26):
    #button = WebDriverWait(driver, 3).until(EC.visibility_of_element_located(
            #(By.XPATH, "//button[@type='button'][@class='slick-next slick-arrow']")))
    #driver.execute_script("arguments[0].scrollIntoView();", button)
    #driver.execute_script("arguments[0].click();", button)
    #time.sleep(2)

#add_to_link_list()


    

In [5]:
import requests


test_url = "https://www.bundestag.de/ajax/filterlist/de/services/opendata/866354-866354?limit=10&offset=00"

req = requests.get(test_url)
soup_links = BeautifulSoup(req.content, 'html.parser')
regex = re.compile('.*XML .*')
documents = soup_links.find_all(title=regex)
for i in documents:
    link_list.append(i['href'])


['/resource/blob/886606/a8b5974c3cd61a9f56f8a972e8e7ad6f/20025-data.xml', '/resource/blob/887038/062d6bc0522f81a988c24eb8f8ee4aee/20026-data.xml', '/resource/blob/886416/db86cf1a7233a5b3fae2c1ed3e2f7445/20024-data.xml', '/resource/blob/885922/ea6510fa33ceba81a3f39fa67157244a/20023-data.xml', '/resource/blob/885462/560c980be301d6511501f0cc7859bac6/20022-data.xml', '/resource/blob/884900/90d7ed39e77924cd38f6bc4e822db6f3/20021-data.xml', '/resource/blob/884842/d4f8f013accfe01ece070441da787aff/20020-data.xml', '/resource/blob/882434/63345d000dff4dac33355f3669e7eec2/20019-data.xml', '/resource/blob/881806/6b8e6220659315227eb3b1dbdbdf8e76/20018-data.xml', '/resource/blob/881518/ff4f6567e6bb232b483190ae0dd9cd54/20017-data.xml']


In [6]:

# Liste mit vollständigen URLs
url_list = []
url_start = 'https://www.bundestag.de'

for i in link_list:
    url_list.append(url_start + i)
    
# soup_links = BeautifulSoup(test_url, 'html.parser')
    

['https://www.bundestag.de/resource/blob/886606/a8b5974c3cd61a9f56f8a972e8e7ad6f/20025-data.xml', 'https://www.bundestag.de/resource/blob/887038/062d6bc0522f81a988c24eb8f8ee4aee/20026-data.xml', 'https://www.bundestag.de/resource/blob/886416/db86cf1a7233a5b3fae2c1ed3e2f7445/20024-data.xml', 'https://www.bundestag.de/resource/blob/885922/ea6510fa33ceba81a3f39fa67157244a/20023-data.xml', 'https://www.bundestag.de/resource/blob/885462/560c980be301d6511501f0cc7859bac6/20022-data.xml', 'https://www.bundestag.de/resource/blob/884900/90d7ed39e77924cd38f6bc4e822db6f3/20021-data.xml', 'https://www.bundestag.de/resource/blob/884842/d4f8f013accfe01ece070441da787aff/20020-data.xml', 'https://www.bundestag.de/resource/blob/882434/63345d000dff4dac33355f3669e7eec2/20019-data.xml', 'https://www.bundestag.de/resource/blob/881806/6b8e6220659315227eb3b1dbdbdf8e76/20018-data.xml', 'https://www.bundestag.de/resource/blob/881518/ff4f6567e6bb232b483190ae0dd9cd54/20017-data.xml']


In [9]:
# Parser 
class SpeechHandler (xml.sax.ContentHandler):
    
    def __init__(self):
        self.CurrentData = ""
        self.p = ""
        self.tagesordnungspunkt = ""
        self.vorname = ""
        self.nachname = ""
        self.fraktion = ""
        self.redeID = ""
        self.startReden = False
        

    def startElement(self, tag, attributes):

        self.CurrentData = tag
        if tag == "rede":
            self.redeID = attributes['id']
            self.startReden = True
            self.p = ""
    
    def endElement(self,tag):
        self.CurrentData = tag
        if self.CurrentData == 'rede':
            store.loc[len(store.index)] = [ self.redeID, self.vorname, self.nachname, self.fraktion, self.p]
            self.p = ""
            self.fraktion = ""
            self.vorname = ""
            self.nachname = ""
    
    def characters (self, content):

        if self.CurrentData == "p":
            self.p = self.p + content
        elif self.CurrentData == "vorname" and self.startReden:
            self.vorname = self.vorname + content
        elif self.CurrentData == "nachname" and self.startReden:
            self.nachname = self.nachname + content
        elif self.CurrentData == "fraktion" and self.startReden:
            self.fraktion = self.fraktion + content

In [16]:
# parsing
count = 1
for url in url_list:
    # hier kann man jetzt alle Files einzeln parsen
    file = url
    store = pd.DataFrame({ 'redeID': [''], 'vorname': [''], 'nachname' : [''], 'fraktion': [''], 'inhalt' : ['']})
    parser  = xml.sax.make_parser()
    Handler = SpeechHandler()
    parser.setContentHandler(Handler)
    parser.parse(file)
    store.to_csv('bundestag ' + str(count) + '.csv')
    count = count + 1

In [13]:
driver.quit()